In [294]:
import numpy as np
import pandas as pd

In [295]:
#引入數據

#引入統整.xlsx的data
data = pd.read_excel("統整.xlsx", sheet_name = "data(刪不要的)", usecols = ["review id", "IRB/REC案號", "申請案送出日", "計畫服務通過日"])

#引入統整的data的作業時間表
time = pd.read_excel("統整.xlsx", sheet_name = "全部時間統整", usecols = [1, 4, 5])

#引入大表
temp = pd.read_excel("106-109 大表.xlsx", sheet_name = 0, usecols = ["案別", "別", "新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症", "試驗階段"])

In [296]:
#整合部分data以及大表

merged_data = data.merge(temp, left_on = "IRB/REC案號", right_on = "案別").drop("案別", 1)
merged_data.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段
0,2313,106019-I,2017-01-11 10:17:53,2017-03-03 09:29:04,I,新藥,Ⅲ
1,2347,106031-I,2017-03-06 19:48:52,2017-03-27 09:59:59,I,新藥,Ⅲ
2,2348,106032-I,2017-03-06 19:49:20,2017-03-27 18:05:07,I,新藥,Ⅲ
3,2365,106081-E,2017-01-03 19:27:48,2017-06-30 11:37:36,E,NaN,藥品其他研究
4,2371,106047-E,2017-01-06 10:02:13,2017-04-26 13:20:41,E,NaN,非藥品研究


In [321]:
#計算計畫主持人作業總時間，基本上，計畫主持人作業總時間 = 所有的 "PI再送出 - PI代辦 or 發會議決議" 的總和

#將時間表排序：revivew id由小至大，時間由早至晚
time_temp = time.sort_values(["rid", "date"], ascending = [True, True]).reset_index(drop = True)

#取出所有review id
review_id = sorted(list(pd.unique(time_temp["rid"])))        

#開始計算計畫主持人作業總時間
time_pi_time = dict()    #創立一個空的計畫主持人作業總時間字典，會將review id當成key然後計畫主持人作業總時間當成value
for i in review_id:      #for迴圈 一個loop代表一個review id
    
    #設定初始值以及參數
    total_pi_time = pd.to_timedelta(0)    #將每一個review id的計畫主持人作業總時間設定為0
    j = 0                                 #用於確認是否使用最早的PI待辦or發會議決議
    
    #開始檢查每一列數據並計算每次的計畫主持人作業時間 i.e. PI再送出 - PI代辦 or 發會議決議，最後再算出每一個review id的計畫主持人作業總時間
    for index, row in time_temp[time_temp["rid"] == i].iterrows():
        if j == 0 and row["type.1"] == "PI待辦" or row["type.1"] == "發會議決議":     # j == 0 代表還未出現過PI待辦or發會議決議
            temp_time_1 = row["date"]
            j += 1            # j會變成1，代表已得到PI待辦or發會議決議
            
        elif j == 1 and row["type.1"] == "PI再送出":        # j == 1 代表已得到PI待辦or發會議決議，找尋一筆PI再送出
            temp_time_2 = row["date"]
            j = 0             # j會變成0，代表已得到一筆計畫主持人作業時間
            
            bdate = len(pd.bdate_range(start = temp_time_1.date(), end = temp_time_2.date()))  #計算PI再送出到PI代辦之間有幾個工作天
            date = len(pd.date_range(start = temp_time_1.date(), end = temp_time_2.date()))    #計算PI再送出到PI代辦之間有幾天
            
            #確認PI再送出到PI代辦之間有無假日, 如果有，將假日剔除，然後將每一筆計畫主持人作業時間加進計畫主持人作業總時間
            if date - bdate != 0: 
                total_pi_time += row["date"] - temp_time_1 - pd.to_timedelta("1day") * (date - bdate)
            else:
                total_pi_time += row["date"] - temp_time_1
                
        else:        #這個條件會讓for迴圈跳過 1. 連續出現的PI代辦or發會議決議  2. 無用的數據
            pass
        
    time_pi_time[i] = [total_pi_time]    #將每一個review id的計畫主持人作業總時間存進計畫主持人作業總時間字典
    
time_pi_time_df = pd.DataFrame(time_pi_time, index = ["計畫主持人作業總時間"]).transpose().reset_index()     #將計畫主持人作業總時間字典存進DataFrame
time_pi_time_df = time_pi_time_df[time_pi_time_df["index"].isin(review_id)]
time_pi_time_df.head()

In [361]:
#將所有數據結合 i.e. data，大表，計畫主持人作業總時間

final_df = merged_data.merge(time_pi_time_df, left_on = "review id", right_on = "index", how = "outer").drop(["index"], 1)

#計算每一筆review id的審查會作業天數和研究計畫審查天數
final_df["審查會作業天數"] = final_df["計畫服務通過日"] - final_df["申請案送出日"] - final_df["計畫主持人作業總時間"]
final_df["研究計畫審查天數"] = final_df["計畫服務通過日"] - final_df["申請案送出日"]
final_df.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
0,2313,106019-I,2017-01-11 10:17:53,2017-03-03 09:29:04,I,新藥,Ⅲ,26 days 13:45:39,24 days 09:25:32,50 days 23:11:11
1,2347,106031-I,2017-03-06 19:48:52,2017-03-27 09:59:59,I,新藥,Ⅲ,1 days 02:35:49,19 days 11:35:18,20 days 14:11:07
2,2348,106032-I,2017-03-06 19:49:20,2017-03-27 18:05:07,I,新藥,Ⅲ,3 days 00:50:01,17 days 21:25:46,20 days 22:15:47
3,2365,106081-E,2017-01-03 19:27:48,2017-06-30 11:37:36,E,NaN,藥品其他研究,115 days 08:39:55,62 days 07:29:53,177 days 16:09:48
4,2371,106047-E,2017-01-06 10:02:13,2017-04-26 13:20:41,E,NaN,非藥品研究,58 days 12:11:29,51 days 15:06:59,110 days 03:18:28


In [337]:
#提取簡易審查的資料

expedited_review = final_df[(final_df["別"] == "I") | (final_df["別"] == "C") | (final_df["別"] == "E") | (["別"] == "J")]
expedited_review.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
0,2313,106019-I,2017-01-11 10:17:53,2017-03-03 09:29:04,I,新藥,Ⅲ,26 days 13:45:39,24 days 09:25:32,50 days 23:11:11
1,2347,106031-I,2017-03-06 19:48:52,2017-03-27 09:59:59,I,新藥,Ⅲ,1 days 02:35:49,19 days 11:35:18,20 days 14:11:07
2,2348,106032-I,2017-03-06 19:49:20,2017-03-27 18:05:07,I,新藥,Ⅲ,3 days 00:50:01,17 days 21:25:46,20 days 22:15:47
3,2365,106081-E,2017-01-03 19:27:48,2017-06-30 11:37:36,E,NaN,藥品其他研究,115 days 08:39:55,62 days 07:29:53,177 days 16:09:48
4,2371,106047-E,2017-01-06 10:02:13,2017-04-26 13:20:41,E,NaN,非藥品研究,58 days 12:11:29,51 days 15:06:59,110 days 03:18:28


In [338]:
#提取一般審查的資料

full_board_review = final_df[final_df["別"] == "F"]
full_board_review.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
6,2378,106017-F,2017-01-10 08:03:57,2017-04-24 00:00:00,F,NaN,藥品其他研究,34 days 10:33:34,69 days 05:22:29,103 days 15:56:03
16,2426,106141-F,2017-03-08 16:08:39,2017-12-29 13:33:52,F,NaN,非藥品研究,173 days 13:16:15,122 days 08:08:58,295 days 21:25:13
18,2449,106069-F,2017-04-11 17:27:48,2017-07-28 13:37:03,F,NaN,非藥品研究,13 days 12:41:07,94 days 07:28:08,107 days 20:09:15
21,2454,106058-F,2017-03-13 14:17:51,2017-05-01 00:00:00,F,NaN,非藥品研究,13 days 01:16:03,35 days 08:26:06,48 days 09:42:09
24,2473,106061-F,2017-03-22 15:26:01,2017-06-07 00:00:00,F,NaN,非藥品研究,6 days 04:38:00,70 days 03:55:59,76 days 08:33:59


In [339]:
#提取新藥的資料

new_drug = final_df[final_df["新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症"] == "新藥"]
new_drug.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
0,2313,106019-I,2017-01-11 10:17:53,2017-03-03 09:29:04,I,新藥,Ⅲ,26 days 13:45:39,24 days 09:25:32,50 days 23:11:11
1,2347,106031-I,2017-03-06 19:48:52,2017-03-27 09:59:59,I,新藥,Ⅲ,1 days 02:35:49,19 days 11:35:18,20 days 14:11:07
2,2348,106032-I,2017-03-06 19:49:20,2017-03-27 18:05:07,I,新藥,Ⅲ,3 days 00:50:01,17 days 21:25:46,20 days 22:15:47
5,2376,106074-I,2017-04-24 15:21:17,2017-05-26 20:43:46,I,新藥,III,14 days 19:56:59,17 days 09:25:30,32 days 05:22:29
7,2393,106025-I,2017-02-16 09:14:11,2017-03-24 15:31:40,I,新藥,Ⅱ+Ⅲ,8 days 04:42:02,28 days 01:35:27,36 days 06:17:29


In [340]:
#提取新醫療技術的資料

new_tech = final_df[final_df["新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症"] == "新醫療技術"]
new_tech.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
95,2975,106148-F,2017-08-25 09:29:25,2018-04-03 13:14:10,F,新醫療技術,新醫療技術,113 days 12:32:44,107 days 15:12:01,221 days 03:44:45
364,4741,108054-F,2019-04-08 20:22:26,2019-06-04 00:00:00,F,新醫療技術,新醫療技術,6 days 06:00:13,49 days 21:37:21,56 days 03:37:34
420,5181,108131-F,2019-06-20 14:00:24,2020-03-04 00:00:00,F,新醫療技術,非藥品研究,72 days 20:20:46,184 days 13:38:50,257 days 09:59:36


In [341]:
#提取新醫療器材的資料

new_device = final_df[final_df["新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症"] == "新醫療器材"]
new_device.head()

,review id,IRB/REC案號,申請案送出日,計畫服務通過日,別,新藥、新醫療技術、新醫療器材、BA、BE、有許可證用於非許可適應症,試驗階段,計畫主持人作業總時間,審查會作業天數,研究計畫審查天數
110,3042,106122-F,2017-08-16 16:30:45,2017-10-12,F,新醫療器材,第三級新醫療器材(IVD),12 days 23:00:13,43 days 08:29:02,56 days 07:29:15
111,3043,106121-F,2017-08-16 16:32:34,2017-10-17,F,新醫療器材,第三級新醫療器材(IVD),12 days 11:31:42,48 days 19:55:44,61 days 07:27:26
112,3044,106123-F,2017-08-16 17:16:33,2017-10-13,F,新醫療器材,第三級新醫療器材(IVD),11 days 07:38:26,45 days 23:05:01,57 days 06:43:27
113,3045,106124-F,2017-08-16 16:53:32,2017-10-17,F,新醫療器材,第三級新醫療器材(IVD),10 days 09:00:43,50 days 22:05:45,61 days 07:06:28
344,4648,108074-F,2019-04-11 15:16:43,2019-08-27,F,新醫療器材,新醫療器材,35 days 10:53:05,101 days 21:50:12,137 days 08:43:17


In [342]:
#計算結果的function

def final_result(df, year, unit):
    temp_df = df[df["IRB/REC案號"].str.startswith(str(year))]
    print(year, "年\t", "最短\t", unit, ":\t", str(temp_df[unit].min()))
    print(year, "年\t", "平均\t", unit, ":\t", str(temp_df[unit].mean()))
    print(year, "年\t", "最長\t", unit, ":\t", str(temp_df[unit].max()))

In [343]:
print("簡易審查")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(expedited_review, i, "研究計畫審查天數")
    print()

簡易審查
-----------------------------------------------
106 年	 最短	 研究計畫審查天數 :	 8 days 22:51:06
106 年	 平均	 研究計畫審查天數 :	 52 days 04:56:22.356435644
106 年	 最長	 研究計畫審查天數 :	 191 days 22:14:18

107 年	 最短	 研究計畫審查天數 :	 7 days 12:51:23
107 年	 平均	 研究計畫審查天數 :	 57 days 12:00:38.097560976
107 年	 最長	 研究計畫審查天數 :	 299 days 21:23:07

108 年	 最短	 研究計畫審查天數 :	 6 days 06:36:44
108 年	 平均	 研究計畫審查天數 :	 63 days 04:33:48.644628099
108 年	 最長	 研究計畫審查天數 :	 223 days 02:04:01

109 年	 最短	 研究計畫審查天數 :	 7 days 09:57:16
109 年	 平均	 研究計畫審查天數 :	 57 days 19:12:09.222222222
109 年	 最長	 研究計畫審查天數 :	 302 days 09:38:15



In [344]:
print("一般審查")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(full_board_review, i, "研究計畫審查天數")
    print()

一般審查
-----------------------------------------------
106 年	 最短	 研究計畫審查天數 :	 41 days 09:20:25
106 年	 平均	 研究計畫審查天數 :	 103 days 11:11:22.428571429
106 年	 最長	 研究計畫審查天數 :	 295 days 21:25:13

107 年	 最短	 研究計畫審查天數 :	 40 days 01:07:36
107 年	 平均	 研究計畫審查天數 :	 117 days 04:26:42.241379310
107 年	 最長	 研究計畫審查天數 :	 427 days 06:50:29

108 年	 最短	 研究計畫審查天數 :	 43 days 06:36:12
108 年	 平均	 研究計畫審查天數 :	 121 days 11:12:12.413043478
108 年	 最長	 研究計畫審查天數 :	 257 days 09:59:36

109 年	 最短	 研究計畫審查天數 :	 19 days 23:25:04
109 年	 平均	 研究計畫審查天數 :	 120 days 06:02:22.030769230
109 年	 最長	 研究計畫審查天數 :	 437 days 09:26:12



In [345]:
print("新藥")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_drug, i, "研究計畫審查天數")
    print()

新藥
-----------------------------------------------
106 年	 最短	 研究計畫審查天數 :	 20 days 14:11:07
106 年	 平均	 研究計畫審查天數 :	 41 days 06:24:13.933333333
106 年	 最長	 研究計畫審查天數 :	 75 days 02:30:46

107 年	 最短	 研究計畫審查天數 :	 14 days 08:19:23
107 年	 平均	 研究計畫審查天數 :	 46 days 21:18:23.615384615
107 年	 最長	 研究計畫審查天數 :	 98 days 04:12:24

108 年	 最短	 研究計畫審查天數 :	 24 days 20:25:25
108 年	 平均	 研究計畫審查天數 :	 78 days 17:39:07.666666667
108 年	 最長	 研究計畫審查天數 :	 180 days 08:24:24

109 年	 最短	 研究計畫審查天數 :	 12 days 09:04:49
109 年	 平均	 研究計畫審查天數 :	 48 days 17:33:30.666666666
109 年	 最長	 研究計畫審查天數 :	 145 days 13:27:57



In [346]:
print("新醫療技術")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_tech, i, "研究計畫審查天數")
    print()

新醫療技術
-----------------------------------------------
106 年	 最短	 研究計畫審查天數 :	 221 days 03:44:45
106 年	 平均	 研究計畫審查天數 :	 221 days 03:44:45
106 年	 最長	 研究計畫審查天數 :	 221 days 03:44:45

107 年	 最短	 研究計畫審查天數 :	 NaT
107 年	 平均	 研究計畫審查天數 :	 NaT
107 年	 最長	 研究計畫審查天數 :	 NaT

108 年	 最短	 研究計畫審查天數 :	 56 days 03:37:34
108 年	 平均	 研究計畫審查天數 :	 156 days 18:48:35
108 年	 最長	 研究計畫審查天數 :	 257 days 09:59:36

109 年	 最短	 研究計畫審查天數 :	 NaT
109 年	 平均	 研究計畫審查天數 :	 NaT
109 年	 最長	 研究計畫審查天數 :	 NaT



In [347]:
print("新醫療器材")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_device, i, "研究計畫審查天數")
    print()

新醫療器材
-----------------------------------------------
106 年	 最短	 研究計畫審查天數 :	 56 days 07:29:15
106 年	 平均	 研究計畫審查天數 :	 59 days 01:11:39
106 年	 最長	 研究計畫審查天數 :	 61 days 07:27:26

107 年	 最短	 研究計畫審查天數 :	 NaT
107 年	 平均	 研究計畫審查天數 :	 NaT
107 年	 最長	 研究計畫審查天數 :	 NaT

108 年	 最短	 研究計畫審查天數 :	 105 days 12:47:42
108 年	 平均	 研究計畫審查天數 :	 121 days 10:45:29.500000
108 年	 最長	 研究計畫審查天數 :	 137 days 08:43:17

109 年	 最短	 研究計畫審查天數 :	 171 days 07:21:21
109 年	 平均	 研究計畫審查天數 :	 199 days 10:49:56.500000
109 年	 最長	 研究計畫審查天數 :	 227 days 14:18:32



In [348]:
print("簡易審查")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(expedited_review, i, "審查會作業天數")
    print()

簡易審查
-----------------------------------------------
106 年	 最短	 審查會作業天數 :	 7 days 10:38:07
106 年	 平均	 審查會作業天數 :	 32 days 19:26:45.762376237
106 年	 最長	 審查會作業天數 :	 85 days 01:25:41

107 年	 最短	 審查會作業天數 :	 7 days 00:33:45
107 年	 平均	 審查會作業天數 :	 33 days 03:17:59.813008130
107 年	 最長	 審查會作業天數 :	 114 days 04:06:45

108 年	 最短	 審查會作業天數 :	 6 days 04:55:17
108 年	 平均	 審查會作業天數 :	 37 days 10:59:13.561983471
108 年	 最長	 審查會作業天數 :	 104 days 20:01:21

109 年	 最短	 審查會作業天數 :	 7 days 01:50:19
109 年	 平均	 審查會作業天數 :	 33 days 09:09:00.251851852
109 年	 最長	 審查會作業天數 :	 111 days 05:43:41



In [349]:
print("一般審查")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(full_board_review, i, "審查會作業天數")
    print()

一般審查
-----------------------------------------------
106 年	 最短	 審查會作業天數 :	 35 days 08:26:06
106 年	 平均	 審查會作業天數 :	 68 days 07:50:08.314285714
106 年	 最長	 審查會作業天數 :	 122 days 08:08:58

107 年	 最短	 審查會作業天數 :	 32 days 21:59:46
107 年	 平均	 審查會作業天數 :	 70 days 16:54:02.482758621
107 年	 最長	 審查會作業天數 :	 167 days 16:16:41

108 年	 最短	 審查會作業天數 :	 37 days 07:06:32
108 年	 平均	 審查會作業天數 :	 80 days 15:47:03.260869565
108 年	 最長	 審查會作業天數 :	 184 days 13:38:50

109 年	 最短	 審查會作業天數 :	 11 days 03:52:32
109 年	 平均	 審查會作業天數 :	 77 days 15:46:02.092307692
109 年	 最長	 審查會作業天數 :	 196 days 22:25:30



In [350]:
print("新藥")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_drug, i, "審查會作業天數")
    print()

新藥
-----------------------------------------------
106 年	 最短	 審查會作業天數 :	 7 days 10:38:07
106 年	 平均	 審查會作業天數 :	 25 days 01:50:05.666666666
106 年	 最長	 審查會作業天數 :	 57 days 16:46:11

107 年	 最短	 審查會作業天數 :	 8 days 14:14:11
107 年	 平均	 審查會作業天數 :	 29 days 09:33:17.538461538
107 年	 最長	 審查會作業天數 :	 52 days 22:50:30

108 年	 最短	 審查會作業天數 :	 18 days 06:22:16
108 年	 平均	 審查會作業天數 :	 49 days 21:32:38.500000
108 年	 最長	 審查會作業天數 :	 96 days 12:30:43

109 年	 最短	 審查會作業天數 :	 8 days 17:48:47
109 年	 平均	 審查會作業天數 :	 27 days 04:31:28.333333333
109 年	 最長	 審查會作業天數 :	 98 days 01:12:35



In [351]:
print("新醫療技術")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_tech, i, "審查會作業天數")
    print()

新醫療技術
-----------------------------------------------
106 年	 最短	 審查會作業天數 :	 107 days 15:12:01
106 年	 平均	 審查會作業天數 :	 107 days 15:12:01
106 年	 最長	 審查會作業天數 :	 107 days 15:12:01

107 年	 最短	 審查會作業天數 :	 NaT
107 年	 平均	 審查會作業天數 :	 NaT
107 年	 最長	 審查會作業天數 :	 NaT

108 年	 最短	 審查會作業天數 :	 49 days 21:37:21
108 年	 平均	 審查會作業天數 :	 117 days 05:38:05.500000
108 年	 最長	 審查會作業天數 :	 184 days 13:38:50

109 年	 最短	 審查會作業天數 :	 NaT
109 年	 平均	 審查會作業天數 :	 NaT
109 年	 最長	 審查會作業天數 :	 NaT



In [352]:
print("新醫療器材")
print("-----------------------------------------------")

for i in range(106, 110):
    final_result(new_device, i, "審查會作業天數")
    print()

新醫療器材
-----------------------------------------------
106 年	 最短	 審查會作業天數 :	 43 days 08:29:02
106 年	 平均	 審查會作業天數 :	 47 days 06:23:53
106 年	 最長	 審查會作業天數 :	 50 days 22:05:45

107 年	 最短	 審查會作業天數 :	 NaT
107 年	 平均	 審查會作業天數 :	 NaT
107 年	 最長	 審查會作業天數 :	 NaT

108 年	 最短	 審查會作業天數 :	 83 days 15:21:30
108 年	 平均	 審查會作業天數 :	 92 days 18:35:51
108 年	 最長	 審查會作業天數 :	 101 days 21:50:12

109 年	 最短	 審查會作業天數 :	 84 days 19:07:57
109 年	 平均	 審查會作業天數 :	 95 days 10:51:36
109 年	 最長	 審查會作業天數 :	 106 days 02:35:15



In [353]:
print("簡易審查")
print("-----------------------------------------------")

for i in range(106,110):
    final_result(expedited_review, i, "計畫主持人作業總時間")
    print()

簡易審查
-----------------------------------------------
106 年	 最短	 計畫主持人作業總時間 :	 0 days 00:20:46
106 年	 平均	 計畫主持人作業總時間 :	 19 days 09:29:36.594059406
106 年	 最長	 計畫主持人作業總時間 :	 115 days 08:39:55

107 年	 最短	 計畫主持人作業總時間 :	 0 days 00:00:00
107 年	 平均	 計畫主持人作業總時間 :	 24 days 08:42:38.284552845
107 年	 最長	 計畫主持人作業總時間 :	 185 days 17:16:22

108 年	 最短	 計畫主持人作業總時間 :	 0 days 00:24:27
108 年	 平均	 計畫主持人作業總時間 :	 25 days 17:34:35.082644628
108 年	 最長	 計畫主持人作業總時間 :	 133 days 13:18:39

109 年	 最短	 計畫主持人作業總時間 :	 0 days 01:53:09
109 年	 平均	 計畫主持人作業總時間 :	 24 days 10:03:08.970370370
109 年	 最長	 計畫主持人作業總時間 :	 191 days 03:54:34



In [354]:
print("一般審查")
print("-----------------------------------------------")

for i in range(106,110):
    final_result(full_board_review, i, "計畫主持人作業總時間")
    print()

一般審查
-----------------------------------------------
106 年	 最短	 計畫主持人作業總時間 :	 0 days 15:35:22
106 年	 平均	 計畫主持人作業總時間 :	 35 days 03:21:14.114285714
106 年	 最長	 計畫主持人作業總時間 :	 173 days 13:16:15

107 年	 最短	 計畫主持人作業總時間 :	 5 days 17:02:34
107 年	 平均	 計畫主持人作業總時間 :	 46 days 11:32:39.758620689
107 年	 最長	 計畫主持人作業總時間 :	 259 days 14:33:48

108 年	 最短	 計畫主持人作業總時間 :	 2 days 01:10:36
108 年	 平均	 計畫主持人作業總時間 :	 40 days 19:25:09.152173913
108 年	 最長	 計畫主持人作業總時間 :	 128 days 00:50:22

109 年	 最短	 計畫主持人作業總時間 :	 1 days 06:12:21
109 年	 平均	 計畫主持人作業總時間 :	 42 days 14:16:19.938461538
109 年	 最長	 計畫主持人作業總時間 :	 240 days 11:00:42



In [355]:
print("新藥")
print("-----------------------------------------------")

for i in range(106,110):
    final_result(new_drug, i, "計畫主持人作業總時間")
    print()

新藥
-----------------------------------------------
106 年	 最短	 計畫主持人作業總時間 :	 1 days 02:35:49
106 年	 平均	 計畫主持人作業總時間 :	 16 days 04:34:08.266666666
106 年	 最長	 計畫主持人作業總時間 :	 35 days 07:42:16

107 年	 最短	 計畫主持人作業總時間 :	 1 days 19:03:09
107 年	 平均	 計畫主持人作業總時間 :	 17 days 11:45:06.076923077
107 年	 最長	 計畫主持人作業總時間 :	 45 days 05:21:54

108 年	 最短	 計畫主持人作業總時間 :	 5 days 14:27:43
108 年	 平均	 計畫主持人作業總時間 :	 28 days 20:06:29.166666666
108 年	 最長	 計畫主持人作業總時間 :	 86 days 05:43:40

109 年	 最短	 計畫主持人作業總時間 :	 3 days 07:46:38
109 年	 平均	 計畫主持人作業總時間 :	 21 days 13:02:02.333333333
109 年	 最長	 計畫主持人作業總時間 :	 53 days 14:31:11



In [356]:
print("新醫療技術")
print("-----------------------------------------------")

for i in range(106,110):
    final_result(new_tech, i, "計畫主持人作業總時間")
    print()

新醫療技術
-----------------------------------------------
106 年	 最短	 計畫主持人作業總時間 :	 113 days 12:32:44
106 年	 平均	 計畫主持人作業總時間 :	 113 days 12:32:44
106 年	 最長	 計畫主持人作業總時間 :	 113 days 12:32:44

107 年	 最短	 計畫主持人作業總時間 :	 NaT
107 年	 平均	 計畫主持人作業總時間 :	 NaT
107 年	 最長	 計畫主持人作業總時間 :	 NaT

108 年	 最短	 計畫主持人作業總時間 :	 6 days 06:00:13
108 年	 平均	 計畫主持人作業總時間 :	 39 days 13:10:29.500000
108 年	 最長	 計畫主持人作業總時間 :	 72 days 20:20:46

109 年	 最短	 計畫主持人作業總時間 :	 NaT
109 年	 平均	 計畫主持人作業總時間 :	 NaT
109 年	 最長	 計畫主持人作業總時間 :	 NaT



In [357]:
print("新醫療器材")
print("-----------------------------------------------")

for i in range(106,110):
    final_result(new_device, i, "計畫主持人作業總時間")
    print()

新醫療器材
-----------------------------------------------
106 年	 最短	 計畫主持人作業總時間 :	 10 days 09:00:43
106 年	 平均	 計畫主持人作業總時間 :	 11 days 18:47:46
106 年	 最長	 計畫主持人作業總時間 :	 12 days 23:00:13

107 年	 最短	 計畫主持人作業總時間 :	 NaT
107 年	 平均	 計畫主持人作業總時間 :	 NaT
107 年	 最長	 計畫主持人作業總時間 :	 NaT

108 年	 最短	 計畫主持人作業總時間 :	 21 days 21:26:12
108 年	 平均	 計畫主持人作業總時間 :	 28 days 16:09:38.500000
108 年	 最長	 計畫主持人作業總時間 :	 35 days 10:53:05

109 年	 最短	 計畫主持人作業總時間 :	 65 days 04:46:06
109 年	 平均	 計畫主持人作業總時間 :	 103 days 23:58:20.500000
109 年	 最長	 計畫主持人作業總時間 :	 142 days 19:10:35

